You can download the `requirements.txt` for this course from the workspace of this lab. `File --> Open...`

# L2: Create Agents to Research and Write an Article

In this lesson, you will be introduced to the foundational concepts of multi-agent systems and get an overview of the crewAI framework.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [14]:
# !pip install crewai==0.100.1
# !pip install crewai_tools 
# !pip install langchain_community

In [15]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import from the crewAI libray.

In [16]:
from crewai import Agent, Task, Crew

- As a LLM for your agents, you'll be using OpenAI's `gpt-3.5-turbo`.

**Optional Note:** crewAI also allow other popular models to be used as a LLM for your Agents. You can see some of the examples at the [bottom of the notebook](#1).

In [17]:
import os
from utils import get_openai_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [18]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	verbose=True
)

### Agent: Writer

In [19]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

### Agent: Editor

In [20]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [21]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

### Task: Write

In [22]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

### Task: Edit

In [23]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution. 

In [27]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=False
)

## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [28]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

# Agent: Content Planner
## Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


# Agent: Content Planner
## Final Answer: 
Content Plan Document:

Title: Unveiling the Latest Trends in Artificial Intelligence

Outline:
I. Introduction
   A. Definition of Artificial Intelligence (AI)
   B. Importance of AI in today's world
II. Key Players in the AI Industry
   A. Overview of leading companies like Google, Amazon, and IBM
   B. Emerging startups making waves in the AI sector
III. Latest Trends in Artificial Intelligence
   A. Machine learning advancements
   B. Natural language processing innovations
   C. Robotics and automation developments
IV. Noteworthy News in AI
   A. Recent breakthroughs and research find

- Display the results of your execution as markdown in the notebook.

In [34]:
txt = ''' 
# Agent: Content Planner
## Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


# Agent: Content Planner
## Final Answer: 
Content Plan Document:

Title: Unveiling the Latest Trends in Artificial Intelligence

Outline:
I. Introduction
   A. Definition of Artificial Intelligence (AI)
   B. Importance of AI in today's world
II. Key Players in the AI Industry
   A. Overview of leading companies like Google, Amazon, and IBM
   B. Emerging startups making waves in the AI sector
III. Latest Trends in Artificial Intelligence
   A. Machine learning advancements
   B. Natural language processing innovations
   C. Robotics and automation developments
IV. Noteworthy News in AI
   A. Recent breakthroughs and research findings
   B. Impact of AI on various industries
V. Target Audience Analysis
   A. Professionals in tech and innovation sectors
   B. Business decision-makers interested in AI applications
VI. SEO Keywords: 
   A. Artificial Intelligence trends
   B. Key players in AI industry
   C. Latest news on AI
VII. Resources:
   A. TechCrunch
   B. MIT Technology Review
   C. Forbes

Conclusion:
This blog article aims to provide a comprehensive overview of the latest trends in Artificial Intelligence, focusing on key players, recent news, and advancements in the field. By targeting professionals in the tech industry and business leaders looking to stay informed about AI developments, this content piece will offer valuable insights and opportunities for engagement with the cutting-edge world of AI.


# Agent: Content Writer
## Task: 1. Use the content plan to craft a compelling blog post on Artificial Intelligence.
2. Incorporate SEO keywords naturally.
3. Sections/Subtitles are properly named in an engaging manner.
4. Ensure the post is structured with an engaging introduction, insightful body, and a summarizing conclusion.
5. Proofread for grammatical errors and alignment with the brand's voice.



# Agent: Content Writer
## Final Answer: 
# Unveiling the Latest Trends in Artificial Intelligence

## Introduction
In today's rapidly evolving technological landscape, Artificial Intelligence (AI) has emerged as a groundbreaking field that encompasses the development of intelligent machines capable of performing tasks that typically require human intelligence. AI technologies are revolutionizing various industries by enabling automation, data analysis, and decision-making processes at unprecedented levels of efficiency and sophistication.

## Key Players in the AI Industry
Leading the vanguard in AI innovation are tech giants such as Google, Amazon, and IBM, investing heavily in research and development to push the boundaries of machine intelligence. Alongside these industry titans, a wave of emerging startups is disrupting the AI sector with novel approaches and specialized solutions tailored to diverse market needs.

## Latest Trends in Artificial Intelligence
The latest trends in Artificial Intelligence are shaping the future of technology across multiple fronts. Machine learning continues to advance, empowering AI systems to learn from data and improve their performance over time. Natural language processing innovations are enhancing human-machine interactions, enabling seamless communication between users and AI platforms. Additionally, robotics and automation developments are streamlining industrial processes and enhancing workforce efficiency through intelligent machines that optimize tasks and operations.

## Noteworthy News in AI
Recent breakthroughs and research findings in Artificial Intelligence are propelling the field forward, uncovering new possibilities in healthcare, finance, transportation, and beyond. The impact of AI on various industries is profound, with AI-driven solutions driving productivity gains, cost savings, and transformative changes in business operations and consumer experiences.

## Target Audience Analysis
This blog post caters to professionals in the tech and innovation sectors who seek to stay updated on the latest developments in AI. Business decision-makers looking to capitalize on AI applications and strategic insights will find valuable information and trends that could shape their future strategies and investments in this dynamic field. 

## Conclusion
In conclusion, this blog article has provided a comprehensive overview of the latest trends in Artificial Intelligence, highlighting key players, recent news, and advancements in the field. By targeting professionals in the tech industry and business leaders interested in AI applications, this content offers valuable insights and opportunities for engagement with the cutting-edge world of AI. Stay informed, stay ahead in the exciting realm of Artificial Intelligence. 

Don't miss out on the dynamic insights and trends unfolding in the realm of AI; stay connected with industry leaders and cutting-edge research from expert sources like TechCrunch, MIT Technology Review, and Forbes.


# Agent: Editor
## Task: Proofread the given blog post for grammatical errors and alignment with the brand's voice.


# Agent: Editor
## Final Answer: 
# Unveiling the Latest Trends in Artificial Intelligence

## Introduction
In today's ever-changing technological landscape, Artificial Intelligence (AI) stands out as a revolutionary field that focuses on creating intelligent machines capable of performing tasks that historically required human intelligence. The impact of AI technologies is felt across diverse industries, enabling automation, data analysis, and decision-making processes with unparalleled efficiency and sophistication.

## Key Players in the AI Industry
Pioneering AI innovation are tech giants like Google, Amazon, and IBM, investing significantly in research and development to advance machine intelligence boundaries. Simultaneously, emerging startups are disrupting the AI sector with innovative approaches and customized solutions to meet varied market demands.

## Latest Trends in Artificial Intelligence
The ongoing trends in Artificial Intelligence are reshaping the technological landscape. Machine learning is progressing, empowering AI systems to enhance performance by learning from data continuously. Innovations in natural language processing are facilitating seamless human-machine interactions, improving communication between users and AI platforms. Furthermore, robotics and automation advancements are optimizing industrial processes and boosting workforce efficiency through intelligent machines that optimize various tasks and operations.

## Noteworthy News in AI
Recent breakthroughs and research findings in Artificial Intelligence are driving the field forward, uncovering new potentials in healthcare, finance, transportation, and other sectors. The significant impact of AI on various industries is evident, with AI-powered solutions boosting productivity, cutting costs, and fundamentally transforming business operations and consumer experiences.

## Target Audience Analysis
This blog post is tailored for professionals in the tech and innovation sectors who are keen on staying abreast of the latest AI developments. Business decision-makers aiming to leverage AI applications and strategic insights will find valuable trends and information that could influence their future strategies and investments in this dynamic field.

## Conclusion
In conclusion, this blog article has presented an inclusive view of the current trends in Artificial Intelligence, focusing on key players, recent news, and advancements within the sector. By addressing professionals in the tech industry and business leaders interested in AI applications, this content provides valuable insights and opportunities for engagement with the cutting-edge world of AI. Stay ahead in the realm of Artificial Intelligence by remaining informed - don't miss out on dynamic insights and trends from industry leaders and expert sources like TechCrunch, MIT Technology Review, and Forbes.
'''

In [35]:
from IPython.display import Markdown
Markdown(txt)

 
# Agent: Content Planner
## Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


# Agent: Content Planner
## Final Answer: 
Content Plan Document:

Title: Unveiling the Latest Trends in Artificial Intelligence

Outline:
I. Introduction
   A. Definition of Artificial Intelligence (AI)
   B. Importance of AI in today's world
II. Key Players in the AI Industry
   A. Overview of leading companies like Google, Amazon, and IBM
   B. Emerging startups making waves in the AI sector
III. Latest Trends in Artificial Intelligence
   A. Machine learning advancements
   B. Natural language processing innovations
   C. Robotics and automation developments
IV. Noteworthy News in AI
   A. Recent breakthroughs and research findings
   B. Impact of AI on various industries
V. Target Audience Analysis
   A. Professionals in tech and innovation sectors
   B. Business decision-makers interested in AI applications
VI. SEO Keywords: 
   A. Artificial Intelligence trends
   B. Key players in AI industry
   C. Latest news on AI
VII. Resources:
   A. TechCrunch
   B. MIT Technology Review
   C. Forbes

Conclusion:
This blog article aims to provide a comprehensive overview of the latest trends in Artificial Intelligence, focusing on key players, recent news, and advancements in the field. By targeting professionals in the tech industry and business leaders looking to stay informed about AI developments, this content piece will offer valuable insights and opportunities for engagement with the cutting-edge world of AI.


# Agent: Content Writer
## Task: 1. Use the content plan to craft a compelling blog post on Artificial Intelligence.
2. Incorporate SEO keywords naturally.
3. Sections/Subtitles are properly named in an engaging manner.
4. Ensure the post is structured with an engaging introduction, insightful body, and a summarizing conclusion.
5. Proofread for grammatical errors and alignment with the brand's voice.



# Agent: Content Writer
## Final Answer: 
# Unveiling the Latest Trends in Artificial Intelligence

## Introduction
In today's rapidly evolving technological landscape, Artificial Intelligence (AI) has emerged as a groundbreaking field that encompasses the development of intelligent machines capable of performing tasks that typically require human intelligence. AI technologies are revolutionizing various industries by enabling automation, data analysis, and decision-making processes at unprecedented levels of efficiency and sophistication.

## Key Players in the AI Industry
Leading the vanguard in AI innovation are tech giants such as Google, Amazon, and IBM, investing heavily in research and development to push the boundaries of machine intelligence. Alongside these industry titans, a wave of emerging startups is disrupting the AI sector with novel approaches and specialized solutions tailored to diverse market needs.

## Latest Trends in Artificial Intelligence
The latest trends in Artificial Intelligence are shaping the future of technology across multiple fronts. Machine learning continues to advance, empowering AI systems to learn from data and improve their performance over time. Natural language processing innovations are enhancing human-machine interactions, enabling seamless communication between users and AI platforms. Additionally, robotics and automation developments are streamlining industrial processes and enhancing workforce efficiency through intelligent machines that optimize tasks and operations.

## Noteworthy News in AI
Recent breakthroughs and research findings in Artificial Intelligence are propelling the field forward, uncovering new possibilities in healthcare, finance, transportation, and beyond. The impact of AI on various industries is profound, with AI-driven solutions driving productivity gains, cost savings, and transformative changes in business operations and consumer experiences.

## Target Audience Analysis
This blog post caters to professionals in the tech and innovation sectors who seek to stay updated on the latest developments in AI. Business decision-makers looking to capitalize on AI applications and strategic insights will find valuable information and trends that could shape their future strategies and investments in this dynamic field. 

## Conclusion
In conclusion, this blog article has provided a comprehensive overview of the latest trends in Artificial Intelligence, highlighting key players, recent news, and advancements in the field. By targeting professionals in the tech industry and business leaders interested in AI applications, this content offers valuable insights and opportunities for engagement with the cutting-edge world of AI. Stay informed, stay ahead in the exciting realm of Artificial Intelligence. 

Don't miss out on the dynamic insights and trends unfolding in the realm of AI; stay connected with industry leaders and cutting-edge research from expert sources like TechCrunch, MIT Technology Review, and Forbes.


# Agent: Editor
## Task: Proofread the given blog post for grammatical errors and alignment with the brand's voice.


# Agent: Editor
## Final Answer: 
# Unveiling the Latest Trends in Artificial Intelligence

## Introduction
In today's ever-changing technological landscape, Artificial Intelligence (AI) stands out as a revolutionary field that focuses on creating intelligent machines capable of performing tasks that historically required human intelligence. The impact of AI technologies is felt across diverse industries, enabling automation, data analysis, and decision-making processes with unparalleled efficiency and sophistication.

## Key Players in the AI Industry
Pioneering AI innovation are tech giants like Google, Amazon, and IBM, investing significantly in research and development to advance machine intelligence boundaries. Simultaneously, emerging startups are disrupting the AI sector with innovative approaches and customized solutions to meet varied market demands.

## Latest Trends in Artificial Intelligence
The ongoing trends in Artificial Intelligence are reshaping the technological landscape. Machine learning is progressing, empowering AI systems to enhance performance by learning from data continuously. Innovations in natural language processing are facilitating seamless human-machine interactions, improving communication between users and AI platforms. Furthermore, robotics and automation advancements are optimizing industrial processes and boosting workforce efficiency through intelligent machines that optimize various tasks and operations.

## Noteworthy News in AI
Recent breakthroughs and research findings in Artificial Intelligence are driving the field forward, uncovering new potentials in healthcare, finance, transportation, and other sectors. The significant impact of AI on various industries is evident, with AI-powered solutions boosting productivity, cutting costs, and fundamentally transforming business operations and consumer experiences.

## Target Audience Analysis
This blog post is tailored for professionals in the tech and innovation sectors who are keen on staying abreast of the latest AI developments. Business decision-makers aiming to leverage AI applications and strategic insights will find valuable trends and information that could influence their future strategies and investments in this dynamic field.

## Conclusion
In conclusion, this blog article has presented an inclusive view of the current trends in Artificial Intelligence, focusing on key players, recent news, and advancements within the sector. By addressing professionals in the tech industry and business leaders interested in AI applications, this content provides valuable insights and opportunities for engagement with the cutting-edge world of AI. Stay ahead in the realm of Artificial Intelligence by remaining informed - don't miss out on dynamic insights and trends from industry leaders and expert sources like TechCrunch, MIT Technology Review, and Forbes.


## Try it Yourself

- Pass in a topic of your choice and see what the agents come up with!

In [36]:
topic = "2008 Economic crisis"
result = crew.kickoff(inputs={"topic": topic})

# Agent: Content Planner
## Task: 1. Prioritize the latest trends, key players, and noteworthy news on 2008 Economic crisis.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


# Agent: Content Planner
## Final Answer: 
Title: Understanding the 2008 Economic Crisis: Lessons Learned and Implications for Today

Outline:
I. Introduction
- Brief overview of the 2008 Economic crisis
- Importance of understanding the causes and consequences for today's economic landscape

II. Key Points
A. Trends in the 2008 Economic Crisis
1. Housing market bubble
2. Subprime mortgage crisis
3. Collapse of financial institutions
4. Global impact on economies

B. Key Players
1. Lehman Brothers
2. Federal Reserve
3. Mortgage lenders
4. Government policymakers

C. Noteworthy News
1. Bailouts and stimulus packages
2. Regulatory refor

In [37]:
txt2 = ''' 
# Agent: Content Planner
## Task: 1. Prioritize the latest trends, key players, and noteworthy news on 2008 Economic crisis.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


# Agent: Content Planner
## Final Answer: 
Title: Understanding the 2008 Economic Crisis: Lessons Learned and Implications for Today

Outline:
I. Introduction
- Brief overview of the 2008 Economic crisis
- Importance of understanding the causes and consequences for today's economic landscape

II. Key Points
A. Trends in the 2008 Economic Crisis
1. Housing market bubble
2. Subprime mortgage crisis
3. Collapse of financial institutions
4. Global impact on economies

B. Key Players
1. Lehman Brothers
2. Federal Reserve
3. Mortgage lenders
4. Government policymakers

C. Noteworthy News
1. Bailouts and stimulus packages
2. Regulatory reforms post-crisis
3. Long-term effects on employment and GDP

III. Audience Analysis
- Target Audience: Individuals interested in economics, finance professionals, students studying economic history
- Interests: Understanding historical financial crises, implications for current economic policies, learning from past mistakes
- Pain Points: Uncertainty about economic stability, desire for insights to navigate volatile markets, need for educational resources on significant financial events

IV. SEO Keywords
- 2008 Economic crisis
- Financial crisis causes
- Subprime mortgage crisis
- Lehman Brothers bankruptcy
- Global economic recession
- Government bailouts
- Regulatory reforms post-crisis

V. Resources
1. Federal Reserve Bank of St. Louis - Economic Research
2. The Balance - Understanding the 2008 Financial Crisis
3. Investopedia - Insights on the 2008 Crisis and Lessons Learned
4. CNBC - Coverage of the 2008 Economic Crisis and its Impact

Call to Action:
In conclusion, understanding the 2008 Economic crisis is essential for making informed decisions in today's economic landscape. By learning from the past, we can better navigate future challenges and contribute to a more stable financial environment. Stay informed, stay prepared.

Note: Please refer to the attached document for a fully detailed content plan with an outline, audience analysis, SEO keywords, and recommended resources.


# Agent: Content Writer
## Task: 1. Use the content plan to craft a compelling blog post on 2008 Economic crisis.
2. Incorporate SEO keywords naturally.
3. Sections/Subtitles are properly named in an engaging manner.
4. Ensure the post is structured with an engaging introduction, insightful body, and a summarizing conclusion.
5. Proofread for grammatical errors and alignment with the brand's voice.



# Agent: Content Writer
## Final Answer: 
# Understanding the 2008 Economic Crisis: Lessons Learned and Implications for Today

## Introduction

The 2008 economic crisis, often referred to as the Great Recession, was a major global financial downturn that had far-reaching implications. Stemming from a combination of factors, this crisis shook the foundations of the financial world and reshaped economic policies and regulations. Understanding the causes and consequences of this crisis is crucial for navigating today's economic landscape effectively.

## Key Points

### Trends in the 2008 Economic Crisis
The crisis was characterized by several key trends that culminated in a perfect storm. The bursting of the housing market bubble, fueled by risky lending practices, particularly in the subprime mortgage sector, played a central role. This, in turn, led to a domino effect, causing the collapse of major financial institutions like Lehman Brothers and triggering a global economic downturn that reverberated across economies worldwide.

### Key Players
Various entities played significant roles in the crisis. From the failure of Lehman Brothers, a renowned investment bank that filed for bankruptcy in September 2008, to the actions of the Federal Reserve in trying to stabilize the financial system, the crisis highlighted the interconnectedness of financial markets and institutions. Mortgage lenders, whose risky lending practices contributed to the crisis, and government policymakers tasked with implementing measures to mitigate the fallout, were also central figures.

### Noteworthy News
The aftermath of the crisis saw unprecedented government interventions in the form of bailouts and stimulus packages aimed at stabilizing financial markets. Regulatory reforms were implemented post-crisis to prevent similar events, emphasizing the need for stricter oversight and risk management. The long-term effects of the crisis were profound, impacting employment rates and GDP growth for years to come.

## Audience Analysis
Our target audience includes individuals interested in economics, finance professionals, and students studying economic history. With a focus on understanding historical financial crises and their implications for current economic policies, our content aims to provide valuable insights for learning from past mistakes and navigating volatile markets effectively. By addressing pain points related to economic uncertainty and the need for educational resources, we strive to offer a comprehensive understanding of significant financial events.

## SEO Keywords
- 2008 Economic crisis
- Financial crisis causes
- Subprime mortgage crisis
- Lehman Brothers bankruptcy
- Global economic recession
- Government bailouts
- Regulatory reforms post-crisis

## Resources
1. [Federal Reserve Bank of St. Louis - Economic Research](link)
2. [The Balance - Understanding the 2008 Financial Crisis](link)
3. [Investopedia - Insights on the 2008 Crisis and Lessons Learned](link)
4. [CNBC - Coverage of the 2008 Economic Crisis and its Impact](link)

## Call to Action
In conclusion, the 2008 Economic crisis serves as a crucial lesson in the intricacies of the financial system and the importance of prudent economic policies. By studying this crisis and its aftermath, we equip ourselves with knowledge to navigate challenges and contribute to a more resilient financial environment. Stay informed, stay prepared.


# Agent: Editor
## Task: Proofread the given blog post for grammatical errors and alignment with the brand's voice.


# Agent: Editor
## Final Answer: 
# Understanding the 2008 Economic Crisis: Lessons Learned and Implications for Today

## Introduction

The 2008 economic crisis, often referred to as the Great Recession, was a major global financial downturn that had far-reaching implications. It stemmed from a complex interplay of factors, shaking the foundations of the financial world and reshaping economic policies and regulations. Understanding the causes and consequences of this crisis is fundamental for effectively navigating today's economic landscape.

## Key Points

### Trends in the 2008 Economic Crisis
Several key trends characterized the crisis, culminating in a perfect storm. The bursting of the housing market bubble, driven by risky lending practices, particularly in the subprime mortgage sector, played a pivotal role. This set off a chain reaction, leading to the collapse of major financial institutions like Lehman Brothers and triggering a global economic downturn that impacted economies worldwide.

### Key Players
Various entities played crucial roles in the crisis. From the fall of Lehman Brothers, a prominent investment bank that declared bankruptcy in September 2008, to the efforts of the Federal Reserve in stabilizing the financial system, the crisis underscored the interconnected nature of financial markets and institutions. Mortgage lenders, whose risky practices added to the crisis, and government policymakers responsible for implementing measures to alleviate the repercussions, were also central figures.

### Noteworthy News
The aftermath of the crisis witnessed unprecedented government interventions through bailouts and stimulus packages aimed at stabilizing financial markets. Post-crisis, regulatory reforms were put in place to prevent similar occurrences, stressing the importance of enhanced oversight and risk management. The enduring effects of the crisis were profound, impacting employment levels and GDP growth for years to follow.

## Audience Analysis

Our target audience comprises individuals interested in economics, finance professionals, and students studying economic history. Focusing on understanding historical financial crises and their implications for current economic policies, our content aims to offer valuable insights for learning from past errors and effectively navigating volatile markets. By addressing concerns related to economic uncertainties and the necessity for educational resources, we aim to provide a comprehensive understanding of significant financial events.

## SEO Keywords
- 2008 Economic crisis
- Financial crisis causes
- Subprime mortgage crisis
- Lehman Brothers bankruptcy
- Global economic recession
- Government bailouts
- Regulatory reforms post-crisis

## Resources
1. [Federal Reserve Bank of St. Louis - Economic Research](link)
2. [The Balance - Understanding the 2008 Financial Crisis](link)
3. [Investopedia - Insights on the 2008 Crisis and Lessons Learned](link)
4. [CNBC - Coverage of the 2008 Economic Crisis and its Impact](link)

## Call to Action

In conclusion, the 2008 Economic crisis stands as a crucial lesson in understanding the intricacies of the financial system and the significance of prudent economic policies. By delving into this crisis and its aftermath, we arm ourselves with knowledge to navigate challenges and contribute to a more resilient financial environment. Stay informed, stay prepared.

'''

In [38]:
Markdown(txt2)

 
# Agent: Content Planner
## Task: 1. Prioritize the latest trends, key players, and noteworthy news on 2008 Economic crisis.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


# Agent: Content Planner
## Final Answer: 
Title: Understanding the 2008 Economic Crisis: Lessons Learned and Implications for Today

Outline:
I. Introduction
- Brief overview of the 2008 Economic crisis
- Importance of understanding the causes and consequences for today's economic landscape

II. Key Points
A. Trends in the 2008 Economic Crisis
1. Housing market bubble
2. Subprime mortgage crisis
3. Collapse of financial institutions
4. Global impact on economies

B. Key Players
1. Lehman Brothers
2. Federal Reserve
3. Mortgage lenders
4. Government policymakers

C. Noteworthy News
1. Bailouts and stimulus packages
2. Regulatory reforms post-crisis
3. Long-term effects on employment and GDP

III. Audience Analysis
- Target Audience: Individuals interested in economics, finance professionals, students studying economic history
- Interests: Understanding historical financial crises, implications for current economic policies, learning from past mistakes
- Pain Points: Uncertainty about economic stability, desire for insights to navigate volatile markets, need for educational resources on significant financial events

IV. SEO Keywords
- 2008 Economic crisis
- Financial crisis causes
- Subprime mortgage crisis
- Lehman Brothers bankruptcy
- Global economic recession
- Government bailouts
- Regulatory reforms post-crisis

V. Resources
1. Federal Reserve Bank of St. Louis - Economic Research
2. The Balance - Understanding the 2008 Financial Crisis
3. Investopedia - Insights on the 2008 Crisis and Lessons Learned
4. CNBC - Coverage of the 2008 Economic Crisis and its Impact

Call to Action:
In conclusion, understanding the 2008 Economic crisis is essential for making informed decisions in today's economic landscape. By learning from the past, we can better navigate future challenges and contribute to a more stable financial environment. Stay informed, stay prepared.

Note: Please refer to the attached document for a fully detailed content plan with an outline, audience analysis, SEO keywords, and recommended resources.


# Agent: Content Writer
## Task: 1. Use the content plan to craft a compelling blog post on 2008 Economic crisis.
2. Incorporate SEO keywords naturally.
3. Sections/Subtitles are properly named in an engaging manner.
4. Ensure the post is structured with an engaging introduction, insightful body, and a summarizing conclusion.
5. Proofread for grammatical errors and alignment with the brand's voice.



# Agent: Content Writer
## Final Answer: 
# Understanding the 2008 Economic Crisis: Lessons Learned and Implications for Today

## Introduction

The 2008 economic crisis, often referred to as the Great Recession, was a major global financial downturn that had far-reaching implications. Stemming from a combination of factors, this crisis shook the foundations of the financial world and reshaped economic policies and regulations. Understanding the causes and consequences of this crisis is crucial for navigating today's economic landscape effectively.

## Key Points

### Trends in the 2008 Economic Crisis
The crisis was characterized by several key trends that culminated in a perfect storm. The bursting of the housing market bubble, fueled by risky lending practices, particularly in the subprime mortgage sector, played a central role. This, in turn, led to a domino effect, causing the collapse of major financial institutions like Lehman Brothers and triggering a global economic downturn that reverberated across economies worldwide.

### Key Players
Various entities played significant roles in the crisis. From the failure of Lehman Brothers, a renowned investment bank that filed for bankruptcy in September 2008, to the actions of the Federal Reserve in trying to stabilize the financial system, the crisis highlighted the interconnectedness of financial markets and institutions. Mortgage lenders, whose risky lending practices contributed to the crisis, and government policymakers tasked with implementing measures to mitigate the fallout, were also central figures.

### Noteworthy News
The aftermath of the crisis saw unprecedented government interventions in the form of bailouts and stimulus packages aimed at stabilizing financial markets. Regulatory reforms were implemented post-crisis to prevent similar events, emphasizing the need for stricter oversight and risk management. The long-term effects of the crisis were profound, impacting employment rates and GDP growth for years to come.

## Audience Analysis
Our target audience includes individuals interested in economics, finance professionals, and students studying economic history. With a focus on understanding historical financial crises and their implications for current economic policies, our content aims to provide valuable insights for learning from past mistakes and navigating volatile markets effectively. By addressing pain points related to economic uncertainty and the need for educational resources, we strive to offer a comprehensive understanding of significant financial events.

## SEO Keywords
- 2008 Economic crisis
- Financial crisis causes
- Subprime mortgage crisis
- Lehman Brothers bankruptcy
- Global economic recession
- Government bailouts
- Regulatory reforms post-crisis

## Resources
1. [Federal Reserve Bank of St. Louis - Economic Research](link)
2. [The Balance - Understanding the 2008 Financial Crisis](link)
3. [Investopedia - Insights on the 2008 Crisis and Lessons Learned](link)
4. [CNBC - Coverage of the 2008 Economic Crisis and its Impact](link)

## Call to Action
In conclusion, the 2008 Economic crisis serves as a crucial lesson in the intricacies of the financial system and the importance of prudent economic policies. By studying this crisis and its aftermath, we equip ourselves with knowledge to navigate challenges and contribute to a more resilient financial environment. Stay informed, stay prepared.


# Agent: Editor
## Task: Proofread the given blog post for grammatical errors and alignment with the brand's voice.


# Agent: Editor
## Final Answer: 
# Understanding the 2008 Economic Crisis: Lessons Learned and Implications for Today

## Introduction

The 2008 economic crisis, often referred to as the Great Recession, was a major global financial downturn that had far-reaching implications. It stemmed from a complex interplay of factors, shaking the foundations of the financial world and reshaping economic policies and regulations. Understanding the causes and consequences of this crisis is fundamental for effectively navigating today's economic landscape.

## Key Points

### Trends in the 2008 Economic Crisis
Several key trends characterized the crisis, culminating in a perfect storm. The bursting of the housing market bubble, driven by risky lending practices, particularly in the subprime mortgage sector, played a pivotal role. This set off a chain reaction, leading to the collapse of major financial institutions like Lehman Brothers and triggering a global economic downturn that impacted economies worldwide.

### Key Players
Various entities played crucial roles in the crisis. From the fall of Lehman Brothers, a prominent investment bank that declared bankruptcy in September 2008, to the efforts of the Federal Reserve in stabilizing the financial system, the crisis underscored the interconnected nature of financial markets and institutions. Mortgage lenders, whose risky practices added to the crisis, and government policymakers responsible for implementing measures to alleviate the repercussions, were also central figures.

### Noteworthy News
The aftermath of the crisis witnessed unprecedented government interventions through bailouts and stimulus packages aimed at stabilizing financial markets. Post-crisis, regulatory reforms were put in place to prevent similar occurrences, stressing the importance of enhanced oversight and risk management. The enduring effects of the crisis were profound, impacting employment levels and GDP growth for years to follow.

## Audience Analysis

Our target audience comprises individuals interested in economics, finance professionals, and students studying economic history. Focusing on understanding historical financial crises and their implications for current economic policies, our content aims to offer valuable insights for learning from past errors and effectively navigating volatile markets. By addressing concerns related to economic uncertainties and the necessity for educational resources, we aim to provide a comprehensive understanding of significant financial events.

## SEO Keywords
- 2008 Economic crisis
- Financial crisis causes
- Subprime mortgage crisis
- Lehman Brothers bankruptcy
- Global economic recession
- Government bailouts
- Regulatory reforms post-crisis

## Resources
1. [Federal Reserve Bank of St. Louis - Economic Research](link)
2. [The Balance - Understanding the 2008 Financial Crisis](link)
3. [Investopedia - Insights on the 2008 Crisis and Lessons Learned](link)
4. [CNBC - Coverage of the 2008 Economic Crisis and its Impact](link)

## Call to Action

In conclusion, the 2008 Economic crisis stands as a crucial lesson in understanding the intricacies of the financial system and the significance of prudent economic policies. By delving into this crisis and its aftermath, we arm ourselves with knowledge to navigate challenges and contribute to a more resilient financial environment. Stay informed, stay prepared.



<a name='1'></a>
 ## Other Popular Models as LLM for your Agents

#### Hugging Face (HuggingFaceHub endpoint)

```Python
from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    huggingfacehub_api_token="<HF_TOKEN_HERE>",
    task="text-generation",
)

### you will pass "llm" to your agent function
```

#### Mistral API

```Python
OPENAI_API_KEY=your-mistral-api-key
OPENAI_API_BASE=https://api.mistral.ai/v1
OPENAI_MODEL_NAME="mistral-small"
```

#### Cohere

```Python
from langchain_community.chat_models import ChatCohere
# Initialize language model
os.environ["COHERE_API_KEY"] = "your-cohere-api-key"
llm = ChatCohere()

### you will pass "llm" to your agent function
```

### For using Llama locally with Ollama and more, checkout the crewAI documentation on [Connecting to any LLM](https://docs.crewai.com/how-to/LLM-Connections/).